# Experiments with EuroSAT satellite images

In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2
import random

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


## Load EuroSAT images

In [2]:
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')

img_dir = dir_test
img_dir = img_dir.iloc[:,1:].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:100, :] # limit sample size when testing
img_dir

(2700, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_1275.jpg,0,AnnualCrop
1,Residential/Residential_504.jpg,7,Residential
2,Pasture/Pasture_787.jpg,5,Pasture
3,Residential/Residential_1575.jpg,7,Residential
4,Pasture/Pasture_473.jpg,5,Pasture
...,...,...,...
2695,HerbaceousVegetation/HerbaceousVegetation_516.jpg,2,HerbaceousVegetation
2696,HerbaceousVegetation/HerbaceousVegetation_855.jpg,2,HerbaceousVegetation
2697,SeaLake/SeaLake_2696.jpg,9,SeaLake
2698,River/River_472.jpg,8,River


In [3]:
# Load images and labels/classes
images = []
classes = []

base_path = 'dataset/EuroSAT/'
for index, row in img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])

# Normalize images
images = np.array(images) / 255.0
labels = pd.get_dummies(classes).values

In [4]:
images.shape

(2700, 224, 224, 3)

In [5]:
# Preprocess the images and labels
y_true = np.argmax(labels, axis=1)

## ConvNeXt

In [6]:
## load model
from tensorflow.keras.models import load_model
model_1 = load_model("model_1_nFull_epoch5_batch32")

In [7]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_1.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 2s 2s/step


In [8]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 90.81%


In [9]:
class_names = list(pd.get_dummies(classes).columns)
class_names

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [10]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.94      0.97      0.96       300
              Forest       0.75      1.00      0.86       300
HerbaceousVegetation       0.94      0.85      0.90       300
             Highway       0.94      0.87      0.91       250
          Industrial       0.98      0.75      0.85       250
             Pasture       0.98      0.84      0.91       200
       PermanentCrop       0.94      0.93      0.93       250
         Residential       0.83      0.89      0.86       300
               River       0.90      0.95      0.92       250
             SeaLake       1.00      0.98      0.99       300

            accuracy                           0.91      2700
           macro avg       0.92      0.90      0.91      2700
        weighted avg       0.92      0.91      0.91      2700



In [11]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[292   0   0   0   0   1   6   0   1   0]
 [  0 300   0   0   0   0   0   0   0   0]
 [  2  40 256   0   0   1   1   0   0   0]
 [  7   3   1 218   0   0   2   3  16   0]
 [  0   0   0   7 188   0   2  49   4   0]
 [  2  22   1   1   0 168   2   0   4   0]
 [  4   3   8   0   0   1 233   1   0   0]
 [  1  27   4   1   0   0   1 266   0   0]
 [  2   1   1   4   3   0   2   0 237   0]
 [  0   3   1   0   0   0   0   0   2 294]]


In [12]:
cm = confusion_matrix(y_true, y_pred_classes)

In [13]:
# ## My matplotlip is broken so im using something else. 
# ## Feel free to use this if you want. 

# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10, 7))
# sns.set(font_scale=1.4)  # Adjust font size
# sns.heatmap(cm_df, annot=True, annot_kws={"size": 12}, cmap="YlGnBu", fmt="g", linewidths=.5, cbar=False)
# plt.ylabel('True label')
# plt.xlabel('Predicted label')
# plt.title('Confusion Matrix Heatmap')

# plt.show()

In [14]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ViT

Load images in 64x64x3 for ViT

In [15]:
# Load images and labels/classes
images_vit = []
# labels.typeof()

base_path = 'dataset/EuroSAT/'
for index, row in img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    # img = cv2.resize(img, (72, 72))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images_vit.append(img)

# Normalize images
images_vit = np.array(images_vit) / 255.0

In [16]:
import tensorflow_addons as tfa
from tensorflow.keras.models import load_model

custom_objects = {
    "AdamW": tfa.optimizers.AdamW
}

model_2 = load_model("model_vit_classifier", custom_objects=custom_objects)

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning:



TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 




In [17]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images_vit), batch_size):
    batch_pred = model_2.predict(images_vit[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 1s 696ms/step


In [18]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 88.30%


In [19]:
class_names = list(pd.get_dummies(classes).columns)
class_names

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [20]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.91      0.88      0.90       300
              Forest       0.95      0.94      0.94       300
HerbaceousVegetation       0.85      0.87      0.86       300
             Highway       0.83      0.76      0.79       250
          Industrial       0.88      0.95      0.92       250
             Pasture       0.83      0.85      0.84       200
       PermanentCrop       0.78      0.82      0.80       250
         Residential       0.89      0.98      0.93       300
               River       0.92      0.78      0.85       250
             SeaLake       0.95      0.94      0.95       300

            accuracy                           0.88      2700
           macro avg       0.88      0.88      0.88      2700
        weighted avg       0.88      0.88      0.88      2700



In [21]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[265   0   6   0   0   6  17   0   5   1]
 [  0 283   2   0   0   9   0   0   0   6]
 [  4   2 262   1   4   2  13   5   1   6]
 [  4   0   1 189  18   1   9  20   8   0]
 [  0   0   0   4 238   0   0   8   0   0]
 [  2   2   7   4   0 170  12   0   2   1]
 [  6   0  23   9   2   2 205   2   1   0]
 [  0   0   0   2   3   0   1 294   0   0]
 [  9   0   7  18   4  10   6   0 196   0]
 [  1  12   0   0   0   5   0   0   0 282]]


In [22]:
cm = confusion_matrix(y_true, y_pred_classes)

In [23]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## ResNet50

In [24]:
## load model
from tensorflow.keras.models import load_model
model_resnet = load_model("model_resnet50_nFull_epoch20_batch32_lr.00001")

In [25]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_resnet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 1s 738ms/step


In [26]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 44.30%


In [27]:
class_names = list(pd.get_dummies(classes).columns)
class_names

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [28]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.52      0.83      0.64       300
              Forest       0.74      0.05      0.09       300
HerbaceousVegetation       0.00      0.00      0.00       300
             Highway       0.23      0.74      0.35       250
          Industrial       0.87      0.81      0.84       250
             Pasture       0.43      0.01      0.03       200
       PermanentCrop       0.50      0.04      0.07       250
         Residential       0.83      0.86      0.85       300
               River       0.00      0.00      0.00       250
             SeaLake       0.33      0.91      0.49       300

            accuracy                           0.44      2700
           macro avg       0.45      0.43      0.34      2700
        weighted avg       0.45      0.44      0.35      2700



c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\Users\Geran\anaconda3\envs\tfgpu_landclass\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [29]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[248   0   0  34   0   0   2   0   0  16]
 [  0  14   0   2   0   2   0   0   0 282]
 [ 87   0   0 147  10   0   3   9   0  44]
 [ 26   0   0 186   5   0   4  14   0  15]
 [  0   0   0  24 203   0   1  22   0   0]
 [ 13   3   0  52   0   3   0   0   0 129]
 [ 63   0   0 164   6   0  10   5   0   2]
 [  0   0   0  37   5   0   0 258   0   0]
 [ 13   2   0 162   5   1   0   1   0  66]
 [ 23   0   0   2   0   1   0   0   0 274]]


In [30]:
cm = confusion_matrix(y_true, y_pred_classes)

In [31]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()


## MobileNet

In [32]:
## load model
from tensorflow.keras.models import load_model
model_mobilenet = load_model("model_MobileNet_nFull_epoch10_batch32")

In [33]:
# Evaluate the model in batches
batch_size = 32
y_pred_batches = []

for i in range(0, len(images), batch_size):
    batch_pred = model_mobilenet.predict(images[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)


1/1 [==============================] - 0s 308ms/step


In [34]:
# Evaluate the performance
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 92.33%


In [35]:
class_names = list(pd.get_dummies(classes).columns)
class_names

['AnnualCrop',
 'Forest',
 'HerbaceousVegetation',
 'Highway',
 'Industrial',
 'Pasture',
 'PermanentCrop',
 'Residential',
 'River',
 'SeaLake']

In [36]:
print("\nClassification Report:")
print(classification_report(y_true, y_pred_classes, target_names=class_names))


Classification Report:
                      precision    recall  f1-score   support

          AnnualCrop       0.98      0.89      0.93       300
              Forest       0.95      0.96      0.96       300
HerbaceousVegetation       0.90      0.92      0.91       300
             Highway       0.79      0.93      0.85       250
          Industrial       0.96      0.94      0.95       250
             Pasture       0.85      0.94      0.89       200
       PermanentCrop       0.90      0.86      0.88       250
         Residential       0.98      0.99      0.98       300
               River       0.96      0.80      0.88       250
             SeaLake       0.96      0.99      0.98       300

            accuracy                           0.92      2700
           macro avg       0.92      0.92      0.92      2700
        weighted avg       0.93      0.92      0.92      2700



In [37]:
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Confusion Matrix:
[[266   1   1   6   0  11  12   0   2   1]
 [  0 287   3   0   0   1   0   0   0   9]
 [  0   7 277   3   0   7   4   0   0   2]
 [  2   0   0 232   3   4   4   0   5   0]
 [  0   0   0   6 234   0   3   6   1   0]
 [  0   4   3   6   0 187   0   0   0   0]
 [  1   0  21   6   2   4 216   0   0   0]
 [  0   0   1   1   2   0   0 296   0   0]
 [  2   1   2  35   2   6   0   1 201   0]
 [  0   1   1   0   0   1   0   0   0 297]]


In [38]:
cm = confusion_matrix(y_true, y_pred_classes)

In [39]:
## using this because my matplotlib is broken
## feel free to use the block from above if yours work. 

import plotly.figure_factory as ff

z = cm.tolist()  # Convert the confusion matrix to a nested list
z_text = [["{:.0f}".format(y) for y in x] for x in z]  # Format values as strings

fig = ff.create_annotated_heatmap(
    z=z,
    x=class_names,
    y=class_names,
    annotation_text=z_text,
    colorscale="Viridis",
    showscale=True
)

fig.update_layout(
    title="Confusion Matrix Heatmap",
    xaxis_title="Predicted label",
    yaxis_title="True label",
    font=dict(size=12)
)

fig.show()
